In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.models import googlenet
import torchvision.transforms as transforms
import torch.nn.functional as F
from torchvision.transforms import functional as TF
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image

In [2]:
class EmotionDataset(Dataset):
    def __init__(self, csv, n_mfcc):
        file_list = pd.read_csv(csv)
        self.images = []
        self.emotions = []
        emotion_to_int = {'anger': 0, 'angry': 0, 'disgust': 1, 'fear': 2, 'happiness': 3,
                               'neutral': 4, 'sad': 5, 'sadness': 5, 'surprise': 6}

        for i in tqdm(range(len(file_list))):
            name = "datasets/emotion_audio_data/{}.wav".format(file_list.iloc[i, 1])
            y, sr = librosa.load(name, res_type="kaiser_fast", duration=3.0, sr=16000)
            
            # 데이터 길이가 3초보다 짧은 경우 0으로 패딩합니다.
            if len(y) < sr * 3:
                pad_length = sr * 3 - len(y)
                y = np.pad(y, (0, pad_length), mode='constant')

            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
            
            # MFCC를 이미지로 변환하여 크기를 조정합니다.
            mfcc = (mfcc - mfcc.min()) / (mfcc.max() - mfcc.min())  # 정규화
            mfcc = Image.fromarray(mfcc)
            mfcc = mfcc.resize((224, 224), resample=Image.BILINEAR)  # 크기 조정
            mfcc = np.array(mfcc).astype(np.float32)
            mfcc = np.stack([mfcc] * 3, axis=0)  # 채널 수를 3으로 맞춥니다.
            
            mfcc = torch.from_numpy(mfcc)
            self.images.append(mfcc)

            emotion = file_list.iloc[i, 3]
            self.emotions.append(emotion_to_int[emotion])
            
        self.len = len(file_list)
        self.n_mfcc = n_mfcc

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.emotions[idx]
        return image, label

In [20]:
class EmotionDataset(Dataset):
    def __init__(self, csv, n_mfcc):
        file_list = pd.read_csv(csv)
        self.images = []
        self.emotions = []
        emotion_to_int = {'anger': 0, 'angry': 0, 'disgust': 1, 'fear': 2, 'happiness': 3,
                               'neutral': 4, 'sad': 5, 'sadness': 5, 'surprise': 6}
        
        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])

        for i in tqdm(range(len(file_list))):
            name = "datasets/emotion_audio_data/{}.wav".format(file_list.iloc[i, 1])
            y, sr = librosa.load(name, res_type="kaiser_fast", duration=3.0, sr=16000)
            
            # 데이터 길이가 3초보다 짧은 경우 0으로 패딩합니다.
            if len(y) < sr * 3:
                pad_length = sr * 3 - len(y)
                y = np.pad(y, (0, pad_length), mode='constant')

            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
            
            # MFCC를 이미지로 변환하여 크기를 조정합니다.
            image = Image.fromarray(mfcc,"RGB") 
            image = transform(image)
            
            self.images.append(image)

            emotion = file_list.iloc[i, 3]
            self.emotions.append(emotion_to_int[emotion])
            
        self.len = len(file_list)
        self.n_mfcc = n_mfcc

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.emotions[idx]
        return image, label

In [17]:
class EmotionDataset(Dataset):
    def __init__(self, csv, n_mels=128):
        file_list = pd.read_csv(csv)
        self.images = []
        self.emotions = []
        emotion_to_int = {'anger': 0, 'angry': 0, 'disgust': 1, 'fear': 2, 'happiness': 3,
                               'neutral': 4, 'sad': 5, 'sadness': 5, 'surprise': 6}

        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        
        for i in tqdm(range(len(file_list))):
            name = "datasets/emotion_audio_data/{}.wav".format(file_list.iloc[i, 1])
            y, sr = librosa.load(name, res_type="kaiser_fast", duration=3.0, sr=16000)
            
            # 데이터 길이가 3초보다 짧은 경우 0으로 패딩합니다.

            mel_spectrogram = librosa.feature.melspectrogram(y, sr=sr, n_mels=n_mels)
            mel_spectrogram = librosa.power_to_db(mel_spectrogram).astype(np.float32)
            
            # mel_spec을 이미지로 변환하여 크기를 조정합니다.
            #mel_spectrogram = np.stack([mel_spectrogram] * 3, axis=0) 
            image = Image.fromarray(mel_spectrogram,"RGB") 
            image = transform(image)
            
            self.images.append(image)

            emotion = file_list.iloc[i, 3]
            self.emotions.append(emotion_to_int[emotion])
            
        self.len = len(file_list)
        self.n_mfcc = n_mfcc

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.emotions[idx]
        return image, label

In [3]:
n_mfcc = 40
n_mels = 128

In [19]:
dataset = EmotionDataset(csv='datasets/emotion_train.csv', n_mels=n_mels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

100%|████████████████████████████████████████████████████████████████████████████| 35179/35179 [19:38<00:00, 29.85it/s]


In [21]:
dataset = EmotionDataset(csv='datasets/emotion_train.csv', n_mfcc=n_mfcc)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

100%|████████████████████████████████████████████████████████████████████████████| 35179/35179 [20:28<00:00, 28.63it/s]


In [22]:
test_dataset = EmotionDataset(csv='datasets/emotion_test.csv', n_mfcc=n_mfcc)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

100%|██████████████████████████████████████████████████████████████████████████████| 8793/8793 [05:16<00:00, 27.76it/s]


In [20]:
test_dataset = EmotionDataset(csv='datasets/emotion_test.csv', n_mels=n_mels)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

100%|██████████████████████████████████████████████████████████████████████████████| 8793/8793 [05:08<00:00, 28.51it/s]


In [6]:
model = googlenet(pretrained=True)
num_features = model.fc.in_features
num_classes = 7
model.fc = nn.Linear(num_features, num_classes) 

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [24]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [25]:
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy}%')
    return accuracy

In [26]:
num_epochs = 60

In [27]:
for epoch in range(num_epochs):
    running_loss = 0.0
    loop = tqdm(dataloader, total=len(dataloader), leave=True)
    model.train()
    
    for mfccs, labels in loop:
        mfccs = mfccs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(mfccs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        loop.set_description(f'Epoch [{epoch+1}/{num_epochs}]')
        loop.set_postfix(loss=running_loss / (len(dataloader)))
    test(model,test_loader)
    torch.save(model.state_dict(), "result/model_googlenet_40_{}.pth".format(epoch))
    
print('Training finished!')

100%|████████████████████████████████████████████████████████████████████████████████| 550/550 [00:20<00:00, 26.80it/s]


Test Accuracy: 31.650176276583647%


100%|████████████████████████████████████████████████████████████████████████████████| 550/550 [00:10<00:00, 51.75it/s]


Test Accuracy: 31.695667007847153%


100%|████████████████████████████████████████████████████████████████████████████████| 550/550 [00:11<00:00, 49.69it/s]


...

Test Accuracy: 30.44467189810076%


100%|████████████████████████████████████████████████████████████████████████████████| 550/550 [00:09<00:00, 57.67it/s]


Test Accuracy: 26.28226998749005%


Epoch [19/60]:   4%|█▉                                                  | 83/2199 [00:06<02:35, 13.61it/s, loss=0.0326]


KeyboardInterrupt: 